In [1]:
import numpy as np
import pandas as pd
from fredmd import FredMD
import sklearn.pipeline as skpipe
import sklearn.decomposition as skd
import sklearn.preprocessing as skp
import math
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg
from sklearn.utils.extmath import randomized_svd

In [2]:
from datetime import datetime

# Data Loaders

In [730]:
class SPCAData:
    
    def __init__(self, Nfactor=None, vintage=None, maxfactor=8, standard_method=2, ic_method=2,
                 target=None, train_test_split=[('1960-01-01', '1984-12-01'),('1985-01-01', '2019-12-01')], 
                 nlags=1, drop_cols=["ACOGNO", "ANDENOx", "TWEXAFEGSMTHx", "UMCSENTx", "VXOCLSx"]) -> None:
        """
        Create fredmd object
        Auguments:
        1) Nfactor = None: Number of factors to estimate. If None then estimate number of true factors via information critea
        2) vintage = None: Vinatege of data to use in "year-month" format (e.g. "2020-10"). If None use current vintage
        3) maxfactor = 8: Maximimum number of factors to test against information critea. If Nfactor is a number, then this is ignored
        4) standard_method = 2: method to standardize data before factors are estimate. 0 = Identity transform, 1 = Demean only, 2 = Demean and stardize to unit variance. Default = 2.
        5) ic_method = 2: information critea penalty term. Se
        e http://www.columbia.edu/~sn2294/pub/ecta02.pdf page 201, equation 9 for options.
        """
        self.drop_cols = drop_cols
        self.train_test_split = [[datetime.strptime(x, '%Y-%m-%d') for x in split] for split in train_test_split]
        # Make sure arguments are valid
        if standard_method not in [0, 1, 2]:
            raise ValueError(f"standard_method must be in [0, 1, 2], got {standard_method}")
        if ic_method not in [1, 2, 3]:
            raise ValueError(f"ic_method must be in [1, 2, 3], got {ic_method}")
        # Download data
        self.rawseries, self.transforms, self.target, self.train_mask, self.test_mask = self.download_data(vintage, 
                                                                       target if target is not None else "UNRATE",
                                                                         self.train_test_split)
        

        self.target_name = target
        self.standard_method = standard_method
        self.ic_method = ic_method
        self.maxfactor = maxfactor
        self.Nfactor = Nfactor

        self.nlags = nlags

    @staticmethod
    def download_data(vintage, tgt, train_test_split):
        if vintage is None:
            url = 'https://s3.amazonaws.com/files.fred.stlouisfed.org/fred-md/monthly/current.csv'
        else:
            url = f'https://s3.amazonaws.com/files.fred.stlouisfed.org/fred-md/monthly/{vintage}.csv'
        transforms = pd.read_csv(
            url, header=0, nrows=1, index_col=0).transpose()
        transforms.index.rename("series", inplace=True)
        transforms.columns = ['transform']
        transforms = transforms.to_dict()['transform']
        data = pd.read_csv(url, names=transforms.keys(), skiprows=2, index_col=0,
                           skipfooter=1, engine='python', parse_dates=True, infer_datetime_format=True)
        
        train_mask = (data.index > train_test_split[0][0]) & (data.index <= train_test_split[0][1])
        test_mask = (data.index > train_test_split[1][0]) & (data.index <= train_test_split[1][1])
        
        target = None
        if "FB-yeild" in tgt:
            bond_data = pd.read_csv("data/bond_data.csv", engine='python', parse_dates=True, infer_datetime_format=True,
                       skiprows=range(1,45), index_col=4)
            if tgt == 'FB-yeild-1':
                bond_data = bond_data.loc[bond_data['TTERMTYPE'] == 5001]
            elif tgt == 'FB-yeild-2':
                bond_data = bond_data.loc[bond_data['TTERMTYPE'] == 5002]
            elif tgt == 'FB-yeild-3':
                bond_data = bond_data.loc[bond_data['TTERMTYPE'] == 5003]
            elif tgt == 'FB-yeild-4':
                bond_data = bond_data.loc[bond_data['TTERMTYPE'] == 5004]
            elif tgt == 'FB-yeild-5':
                bond_data = bond_data.loc[bond_data['TTERMTYPE'] == 5005]
            bond_data.index = bond_data.index.to_period('M') 
            intersection = bond_data.index.intersection(data.index.to_period('M'))
            target = bond_data[bond_data.index.isin(intersection)]["TMYTM"]
            
        return data, transforms, target, train_mask, test_mask

    @staticmethod
    def factor_standardizer_method(code):
        """
        Outputs the sklearn standard scaler object with the desired features
        codes:
        0) Identity transform
        1) Demean only
        2) Demean and standardized
        """
        if code == 0:
            return skp.StandardScaler(with_mean=False, with_std=False)
        elif code == 1:
            return skp.StandardScaler(with_mean=True, with_std=False)
        elif code == 2:
            return skp.StandardScaler(with_mean=True, with_std=True)
        else:
            raise ValueError("standard_method must be in [0, 1, 2]")

    @staticmethod
    def data_transforms(series, transform):
        """
        Transforms a single series according to its transformation code
        Inputs:
        1) series: pandas series to be transformed
        2) transfom: transform code for the series
        Returns:
        transformed series
        """
        if transform == 1:
            # level
            return series
        elif transform == 2:
            # 1st difference
            return series.diff()
        elif transform == 3:
            # second difference
            return series.diff().diff()
        elif transform == 4:
            # Natural log
            return np.log(series)
        elif transform == 5:
            # log 1st difference
            return np.log(series).diff()
        elif transform == 6:
            # log second difference
            return np.log(series).diff().diff()
        elif transform == 7:
            # First difference of percent change
            return series.pct_change().diff()
        else:
            raise ValueError("Transform must be in [1, 2, ..., 7]")

    def apply_transforms(self):
        """
        Apply the transformation to each series to make them stationary and drop the first 2 rows that are mostly NaNs
        Save results to self.series
        """
        self.series = pd.DataFrame({key: self.data_transforms(
            self.rawseries[key], value) for (key, value) in self.transforms.items()})

    def remove_outliers(self):
        """
        Removes outliers from each series in self.series
        Outlier definition: a data point x of a series X is considered an outlier if abs(x-median)>10*interquartile_range.
        """
        Z = abs((self.series - self.series.median()) /
                (self.series.quantile(0.75) - self.series.quantile(0.25))) > 10
        for col, _ in self.series.iteritems():
            self.series[col][Z[col]] = np.nan
        

    def get_data(self):
        """
        """
        # Define our estimation pipelines
        self.apply_transforms()
        self.remove_outliers()
        
        self.series = self.series.loc[self.series.index >= '1960-01-01']
        self.series = self.series.loc[self.series.index < '2020-01-01']
        
        pipe = skpipe.Pipeline([('Standardize', self.factor_standardizer_method(self.standard_method))])

        actual_data = self.series.to_numpy(copy=True)
        intial_nas = self.series.isna().to_numpy(copy=True)
        working_data = self.series.fillna(value=self.series.mean(), axis='index').to_numpy(copy=True)

        last_timestep = np.sum(self.train_mask) + np.sum(self.test_mask)     

        if self.target is None:
            idx = self.series.columns.get_loc(self.target_name)
            target_data = np.copy(working_data[:,idx])

        else:
            target_data = self.target
            target_data = target_data.loc[target_data.index >= '1960-01-01']
            target_data = target_data.loc[target_data.index < '2020-01-01'].to_numpy(copy=True)
        
        bad_cols = np.isin(self.series.columns.to_numpy(), self.drop_cols)
        
        return working_data[:-1,~bad_cols], target_data[1:], np.sum(self.train_mask), working_data[:last_timestep]
        


In [623]:
class ChenZData:
    
    def __init__(self, start_date='1976-03', end_date='2019-09', tgt_factor='mkt'):
        chen_z_port_data = pd.read_csv("data/allportretbase.csv", engine='python', 
                                       parse_dates=True, infer_datetime_format=True,
                               index_col=0)
        chen_z_port_data = chen_z_port_data.loc[chen_z_port_data.date >= start_date]
        chen_z_port_data = chen_z_port_data.loc[chen_z_port_data.date < end_date]

        chen_z_port_data['port_id'] = chen_z_port_data.signalname.add(chen_z_port_data.port.astype(str))
        chen_z_port_data.port_id = chen_z_port_data.port_id
        portfolios = sorted(chen_z_port_data.port_id.unique())
        dates = chen_z_port_data.date.unique()
        portfolios = [p for p in portfolios if 
                     np.sum(chen_z_port_data['port_id'] == p) == len(dates)]
        return_panel = np.zeros((len(dates), len(portfolios)), dtype=float)
        for i, pname in enumerate(portfolios):
            return_panel[:, i] = chen_z_port_data.loc[chen_z_port_data['port_id'] == pname].ret.to_numpy()
        tgt = None
        if tgt_factor in {'mkt', 'SMB', 'HML', 'CMA', 'RMW'}:
            int_start_date = int(start_date.replace('-', ''))
            int_end_date = int(end_date.replace('-', ''))
            ff_factors = pd.read_csv('data/F-F_Research_Data_5_Factors_2x3.CSV', skiprows=0,
                                       engine='python', parse_dates=True, infer_datetime_format=True)
            ff_factors = ff_factors.loc[ff_factors.date >= int_start_date] #NOTE not offsetting by one here across data sources
            ff_factors = ff_factors.loc[ff_factors.date < int_end_date]
            if tgt_factor == 'mkt':
                tgt = ff_factors['Mkt-RF'].to_numpy()
            if tgt_factor == 'SMB':
                tgt = ff_factors['SMB'].to_numpy()
            if tgt_factor == 'HML':
                tgt = ff_factors['HML'].to_numpy()
            if tgt_factor == 'CMA':
                tgt = ff_factors['CMA'].to_numpy()
            if tgt_factor == 'RMW':
                tgt = ff_factors['RMW'].to_numpy()
        self.data = return_panel, tgt, int(tgt.shape[0]/2)

# Models


##  SPCA

In [712]:
def scale(X):
    norm = np.std(X, axis=0, keepdims=True)
    return (X - np.mean(X, axis=0, keepdims=True)) / norm

class SPCA:
    
    def __init__(self, data_panel, target_panel, n_train,
                N_factor=6) -> None:
        """
        Auguments:
        1) data_panel: numpy 2d array of data, normalizations and transforms should have already been applied
        2) target_panel: numpy 2d array of target data, normalizations and transforms should have already been applied
        3) n_train: index of start of oos
        4) N_factors: number of factors
        """
        self.n_factors = N_factor
        self.test_start = n_train
        self.data_series = np.copy(data_panel)
        self.tgt_series = np.copy(target_panel)
        
    def fit(self, nlags, true_oos=False, pca=False, raw_data=None,
           stack_lags=False, plot_resids=False):
        T = self.tgt_series.shape[0]
        preds = []
        ar_preds = []
        gts = []
        mean_preds = []
        
        
        lags = np.zeros((T, nlags))
        if stack_lags:
            for t in range(T):
                lags[t,max(0, nlags-t):] = self.tgt_series[max(0, t-nlags):t]
        for t in range(self.test_start, self.tgt_series.shape[0]):
            scaled_data = scale(np.copy(self.data_series[:t,:]))
            scaled_data_ext = scale(np.copy(self.data_series[:t+1,:]))
            assert np.sum(np.isnan(scaled_data)) == 0 and np.sum(np.isnan(scaled_data_ext)) == 0 
            gamma_is = SPCA.get_gamma_is(scaled_data, self.tgt_series[:t])

            if pca:
                gamma_is[:] = 1
            gamma_is = np.diag(gamma_is)
            if true_oos:
                fit_factors = SPCA.fit_factors(scaled_data@gamma_is, self.n_factors)
                test_factors = SPCA.fit_factors(scaled_data_ext@gamma_is, self.n_factors)[t:]
            else:
                factors = SPCA.fit_factors(scaled_data_ext@gamma_is, self.n_factors, raw_data=raw_data)
                fit_factors = factors[:t]
                test_factors = factors[t:]
            
            if stack_lags:
                A_fit = np.concatenate([fit_factors, np.ones((t, 1)), lags[:t]], axis=1)
                A_test = np.concatenate([test_factors, np.ones((1, 1)), lags[t:t+1]], axis=1)
            else:
                A_fit = np.concatenate([fit_factors, np.ones((t, 1))], axis=1)
                A_test = np.concatenate([test_factors, np.ones((1, 1))], axis=1)
            loadings = np.linalg.lstsq(A_fit, self.tgt_series[:t], rcond=None)[0]

            
            sim_ar_model = AutoReg(self.tgt_series[:t], lags=nlags, old_names=False,
                                  trend='n')
            sim_ar_model_fit = sim_ar_model.fit()
            
            gts.append(self.tgt_series[t])
            mean_preds.append(np.mean(self.tgt_series[:t]))
            ar_forcast = sim_ar_model_fit.forecast()
            ar_preds.append(ar_forcast)
            preds.append(A_test@loadings)
            
        preds = np.array(preds).squeeze()
        ar_preds = np.array(ar_preds).squeeze()
        gts = np.array(gts)
        mean_preds = np.array(mean_preds)

        if plot_resids:
            plt.plot(preds - gts, label='factor resids')
            plt.plot(ar_preds - gts, label='ar resids')
            plt.legend()
            plt.show()
        
        print("r2 vs ar model", 1 - np.sum(np.square(preds - gts)) / np.sum(np.square(gts - ar_preds)))
        print("ar r2 vs mean model", 1 - np.sum(np.square(gts - ar_preds)) / np.sum(np.square(gts - mean_preds)))
        print("r2 vs mean model", 1 - np.sum(np.square(preds - gts)) / np.sum(np.square(gts - mean_preds)))
        print("EV", 1 - np.sum(np.square(preds - gts)) / np.sum(np.square(gts)))
        

        
    def fit_ts_reg(self, plot_resids=False):
        self.test_start
        gamma_is = SPCA.get_gamma_is(self.data_series[:self.test_start,:], self.tgt_series[:self.test_start])
        factors = SPCA.fit_factors(self.data_series*gamma_is, self.n_factors)[self.test_start:]
        loadings = np.linalg.lstsq(factors, self.tgt_series[self.test_start:], rcond=None)[0]
        preds = factors.dot(loadings)
        gts = self.tgt_series[self.test_start:]
        if plot_resids:
            plt.plot(preds - gts, label='factor resids')
            plt.legend()
            plt.show()
        
        print("r2 vs mean model", 1 - np.sum(np.square(preds - gts)) / np.sum(np.square(gts - np.mean(gts))))
        print("EV", 1 - np.sum(np.square(preds - gts)) / np.sum(np.square(gts)))
    

    @staticmethod
    def fit_factors(scaled_data, n_factors, raw_data = None):
        T,N = scaled_data.shape
        if raw_data is None:
            fit_pipe = skpipe.Pipeline([('loadings', skd.TruncatedSVD(n_factors, algorithm='arpack'))])
            objective = scaled_data.T.dot(scaled_data)
            loadings = fit_pipe.fit_transform(objective)
            factors = (1/N ) * scaled_data.dot(loadings)
        else:
            evals, evects = np.linalg.eigh(scaled_data.T.dot(scaled_data))
            evects = evects[:,::-1]
            lmbda = evects[:,:n_factors]
            factors = (1/N) * raw_data[:T]@lmbda
            
        return factors
        
    
    @staticmethod
    def get_gamma_is(data, target):
        loadings = []
        T = target.shape[0]
        
        for i in range(data.shape[1]):
            A = np.stack([data[:,i], np.ones(T)], axis=1)
            loading = np.linalg.lstsq(A, target, rcond=None)[0][0]
            loadings.append(loading)
        return np.array(loadings)

    

In [733]:
def scale(X):
    norm = np.std(X, axis=0, keepdims=True)
    return (X - np.mean(X, axis=0, keepdims=True)) / norm

class GX_SPCA:
    
    def __init__(self, data_panel, target_panel, test_start, N_factors=6):
        """
        Auguments:
        1) data_panel: numpy 2d array of data, normalizations and transforms should have already been applied
        2) target_panel: numpy 2d array of target data, normalizations and transforms should have already 
            been applied
        3) n_train: index of start of oos
        4) N_factors: number of factors
        """
        T,N = data_panel.shape
        self.train_data = data_panel[:test_start]
        self.train_target = target_panel[:test_start]
        self.test_data = data_panel[test_start:]
        self.test_target = target_panel[test_start:]
        self.n_factors = N_factors
        
    def fit(self, nlags, true_oos=False, pca=False, raw_data=None,
           stack_lags=False, plot_resids=False, quantile=0.75):
        
        
        betas = GX_SPCA.get_portfolio_loadings(self.train_data, self.train_target, self.n_factors,
                                              quantile=quantile)
        
        train_factors = self.train_data @ betas
        test_factors = self.test_data @ betas
        
        preds = []
        mean_preds = []
        gts = []
        for t in range(self.test_data.shape[0]):
            if t > 0:
                fit_factors = np.concatenate([train_factors, test_factors[:t]], axis=0)
                fit_target = np.concatenate([self.train_target, self.test_target[:t]], axis=0)
            else:
                fit_factors = train_factors
                fit_target = self.train_target
            
            eval_factors = test_factors[t:t+1]
            
            A_fit = np.concatenate([fit_factors, np.ones((fit_factors.shape[0], 1))], axis=1)
            A_test = np.concatenate([eval_factors, np.ones((1, 1))], axis=1)
            
            loadings = np.linalg.lstsq(A_fit, fit_target, rcond=None)[0]
        
            gts.append(self.test_target[t])
            mean_preds.append(np.mean(np.concatenate([self.train_target,
                                                self.test_target[:t]], axis=0)))
        
            preds.append(A_test@loadings)
            
        preds = np.array(preds).squeeze()
        mean_preds = np.array(mean_preds)
        gts = np.array(gts)
        print("r2 vs mean model", 1 - np.sum(np.square(preds - gts)) / 
              np.sum(np.square(gts - np.mean(self.train_target))))
        print("EV", 1 - np.sum(np.square(preds - gts)) / np.sum(np.square(gts)))
    
    
    @staticmethod
    def get_portfolio_loadings(train_data, train_tgt, nfactor, quantile):
        T, N = train_data.shape
        loadings = np.zeros((N, nfactor))
        
        R_k = np.copy(train_data.T)
        G_k = np.copy(train_tgt.T)
        rbar_k = np.mean(R_k, axis=1)
        
        eta_hats = []
        gamma_hats = []
        v_hats = []
        beta_hats = []
        
        for k in range(nfactor):
            vhat, gamma_hat, eta_hat, beta_hat, R_k, rbar_k, G_k = GX_SPCA.get_next_factor_loading(R_k, G_k, rbar_k,
                                                                                                  quantile=quantile)
            eta_hats.append(eta_hat)
            gamma_hats.append(gamma_hat)
            v_hats.append(vhat)
            beta_hats.append(beta_hat)
        
        betas = np.concatenate(beta_hats, axis=1)
        return betas
        
    @staticmethod
    def get_next_factor_loading(train_data, train_tgt, rbar, quantile):
        #train_data: n times T
        #train_tgt: 1 times T
        N,T = train_data.shape 
        cors = (1/T)*(train_data @ train_tgt.T)
        thresh = np.quantile(cors, quantile) # FIXME
        selected_inds = np.argwhere(cors > thresh).squeeze()
        selected_data = train_data[selected_inds,:] #
        
        # algo 1
        psi, singval, xi = randomized_svd(selected_data, n_components=1,
                                           n_iter=10, random_state=None) #numiter?
        vhat = np.sqrt(T) * xi #factors
        gamma_hat = (singval/T)*psi.T @ rbar[selected_inds]
        eta_hat = (1/T) * train_tgt @ vhat.T
        #END algo 1
        
        beta_hat = (1/T) * train_data @ vhat.T
        
        train_data_perp = train_data - beta_hat @ vhat
        rbar_perp = rbar - beta_hat.squeeze() * gamma_hat
        target_perp = train_tgt - eta_hat * vhat.squeeze()
        
        return vhat, gamma_hat, eta_hat, beta_hat, train_data_perp, rbar_perp, target_perp

# Metrics

In [577]:
from statsmodels.tools.eval_measures import bic

In [581]:
def get_lags(target_series_train, max_lags):
    bics = []
    for i in range(max_lags):
        i += 1
        model = AutoReg(target_series_train, lags=i, old_names=False, trend='n')
        model_fit = model.fit()
        bics.append(model_fit.bic)
    return np.argmax(bics) + 1

In [734]:
for tgt in ["VXOCLSx", "UNRATE", "INDPRO", "CPIAUCSL"]:
    spca_data = SPCAData(target=tgt, 
                         drop_cols=["ACOGNO", "ANDENOx", "TWEXAFEGSMTHx", "UMCSENTx", "VXOCLSx"]
#                          drop_cols=["ACOGNO", "ANDENOx", "TWEXAFEGSMTHx", "UMCSENTx"]
                        )
    data_panel, tgt_data, test_start, raw_data = spca_data.get_data()
    opt_lags = get_lags(tgt_data[:test_start], max_lags=5)
    print(tgt, opt_lags)
    spca = SPCA(data_panel, tgt_data, test_start, N_factor=4)
    spca.fit(opt_lags, true_oos=True, stack_lags=False)
    spca.fit(opt_lags, true_oos=True, stack_lags=False, pca=True)
    print("giglio xui")
    for q in range(50):
        print("q", f'{q}/50')
        spca = GX_SPCA(data_panel, tgt_data, test_start, N_factors=5)
        spca.fit(1, true_oos=True, stack_lags=False, quantile=q/50)
    print()

VXOCLSx 1
r2 vs ar model -2.5323516651749176
ar r2 vs mean model 0.77591764844881
r2 vs mean model 0.2084623323618423
EV 0.8833962896867436
r2 vs ar model -2.946555075137859
ar r2 vs mean model 0.77591764844881
r2 vs mean model 0.11564665823682474
EV 0.869723343444657
giglio xui
q 0/50
r2 vs mean model 0.17270075245693906
EV 0.8748970747029159
q 1/50
r2 vs mean model 0.16751550669246296
EV 0.8741129697790364
q 2/50
r2 vs mean model 0.1256254274534434
EV 0.8677784161464885
q 3/50
r2 vs mean model 0.0885146344007236
EV 0.86216657885209
q 4/50
r2 vs mean model 0.08705692593457981
EV 0.8619461464101574
q 5/50
r2 vs mean model 0.09244396336452831
EV 0.8627607659606682
q 6/50
r2 vs mean model 0.10846784319185232
EV 0.8651838725293743
q 7/50
r2 vs mean model 0.10651704656470518
EV 0.8648888760508442
q 8/50
r2 vs mean model 0.10664542571345537
EV 0.8649082893491199
q 9/50
r2 vs mean model 0.10681569192210416
EV 0.8649340367444481
q 10/50
r2 vs mean model 0.08425237327636614
EV 0.86152204625204

r2 vs mean model 0.018645823396677308
EV 0.04927288049803158
q 9/50
r2 vs mean model 0.01864672426125258
EV 0.04927375324754757
q 10/50
r2 vs mean model 0.018639642568521908
EV 0.049266892567182086
q 11/50
r2 vs mean model 0.018639676919109283
EV 0.0492669258457229
q 12/50
r2 vs mean model 0.018639841873773433
EV 0.049267085652321185
q 13/50
r2 vs mean model 0.018639890259852798
EV 0.04926713252832071
q 14/50
r2 vs mean model 0.018638854541390693
EV 0.049266129133569314
q 15/50
r2 vs mean model 0.01863886278702931
EV 0.04926613712187
q 16/50
r2 vs mean model 0.018638846355258143
EV 0.04926612120291751
q 17/50
r2 vs mean model 0.01863883768565766
EV 0.049266112803886375
q 18/50
r2 vs mean model 0.018639059340969988
EV 0.04926632754156379
q 19/50
r2 vs mean model 0.018639037550026938
EV 0.04926630643069363
q 20/50
r2 vs mean model 0.018639119384352587
EV 0.04926638571105402
q 21/50
r2 vs mean model 0.018638963376530704
EV 0.04926623457207624
q 22/50
r2 vs mean model 0.018638913323802964


In [735]:
for i in range(5):
    tgt = f'FB-yeild-{i+1}'
    spca_data = SPCAData(target=tgt, 
                         drop_cols=["ACOGNO", "ANDENOx", "TWEXAFEGSMTHx", "UMCSENTx", "VXOCLSx"]
#                          drop_cols=["ACOGNO", "ANDENOx", "TWEXAFEGSMTHx", "UMCSENTx"]
                        )
    data_panel, tgt_data, test_start, raw_data = spca_data.get_data()
    opt_lags = get_lags(tgt_data[:test_start], max_lags=5)
    print(tgt, opt_lags)
    spca = SPCA(data_panel, tgt_data, test_start, N_factor=4)
    spca.fit(opt_lags, true_oos=True, stack_lags=False)
    spca.fit(opt_lags, true_oos=True, pca=True)
    print("giglio xui")
    for q in range(50):
        print("q", f'{q}/50')
        spca = GX_SPCA(data_panel, tgt_data, test_start, N_factors=5)
        spca.fit(1, true_oos=True, stack_lags=False, quantile=q/50)
    print()

FB-yeild-1 4
r2 vs ar model -102.65667453070951
ar r2 vs mean model 0.9942046744466398
r2 vs mean model 0.3992758253158427
EV 0.672327080847943
r2 vs ar model -110.72040551769886
ar r2 vs mean model 0.9942046744466398
r2 vs mean model 0.3525438790715212
EV 0.6468365248010104
giglio xui
q 0/50
r2 vs mean model 0.6321732235302502
EV 0.6973528037870316
q 1/50
r2 vs mean model 0.662302574950056
EV 0.7221431788065873
q 2/50
r2 vs mean model 0.6610794098254127
EV 0.7211367607882155
q 3/50
r2 vs mean model 0.736605818329772
EV 0.7832797510111134
q 4/50
r2 vs mean model 0.7331369135952845
EV 0.7804255425657723
q 5/50
r2 vs mean model 0.7179096019532007
EV 0.7678965385096559
q 6/50
r2 vs mean model 0.7174288750517177
EV 0.7675009972979084
q 7/50
r2 vs mean model 0.7050900395909658
EV 0.7573486261394786
q 8/50
r2 vs mean model 0.7064749912588634
EV 0.75848816182854
q 9/50
r2 vs mean model 0.7045034716774046
EV 0.7568659991373565
q 10/50
r2 vs mean model 0.704499656843854
EV 0.7568628602993166
q 

r2 vs mean model 0.6608352238748483
EV 0.7922088711026253
q 16/50
r2 vs mean model 0.6608338374510916
EV 0.792208021702635
q 17/50
r2 vs mean model 0.6608313882935749
EV 0.7922065212130764
q 18/50
r2 vs mean model 0.6608305535366269
EV 0.7922060097947312
q 19/50
r2 vs mean model 0.6608274003284733
EV 0.7922040779647535
q 20/50
r2 vs mean model 0.6608273963811193
EV 0.7922040755463858
q 21/50
r2 vs mean model 0.6608270034385505
EV 0.7922038348080023
q 22/50
r2 vs mean model 0.6608281602457192
EV 0.7922045435321518
q 23/50
r2 vs mean model 0.6608269837950196
EV 0.792203822773287
q 24/50
r2 vs mean model 0.6608267347742887
EV 0.7922036702093935
q 25/50
r2 vs mean model 0.6608266176476807
EV 0.7922035984511456
q 26/50
r2 vs mean model 0.6608265261720017
EV 0.7922035424080777
q 27/50
r2 vs mean model 0.6608262824953104
EV 0.7922033931182387
q 28/50
r2 vs mean model 0.6608262806701488
EV 0.7922033920000436
q 29/50
r2 vs mean model 0.6608262658916315
EV 0.7922033829459054
q 30/50
r2 vs mean m

r2 vs mean model 0.6271633741625411
EV 0.8238816030805278
q 32/50
r2 vs mean model 0.6271633451112177
EV 0.8238815893574327
q 33/50
r2 vs mean model 0.6271622922014017
EV 0.8238810919900175
q 34/50
r2 vs mean model 0.6271621840774071
EV 0.8238810409150337
q 35/50
r2 vs mean model 0.6271621052862439
EV 0.8238810036961223
q 36/50
r2 vs mean model 0.627161779634203
EV 0.8238808498665051
q 37/50
r2 vs mean model 0.6271613632600801
EV 0.8238806531821186
q 38/50
r2 vs mean model 0.6271613242062918
EV 0.8238806347341172
q 39/50
r2 vs mean model 0.6271645651046942
EV 0.8238821656508557
q 40/50
r2 vs mean model 0.6271636782795337
EV 0.8238817467375441
q 41/50
r2 vs mean model 0.6271045447732975
EV 0.8238538135957848
q 42/50
r2 vs mean model 0.6270700324172864
EV 0.823837510849774
q 43/50
r2 vs mean model 0.6270597169554499
EV 0.8238326380919057
q 44/50
r2 vs mean model 0.6281229675525706
EV 0.824334890225177
q 45/50
r2 vs mean model 0.6281824671227554
EV 0.8243629962860953
q 46/50
r2 vs mean mo

In [726]:
for tgt in 'mkt', 'SMB', 'HML', 'CMA', 'RMW' :
    print(tgt)
    chen_z_data_panel, tgt_factor, weak_test_start = ChenZData(tgt_factor=tgt).data
    spca = SPCA(chen_z_data_panel, tgt_factor, weak_test_start, N_factor=5)
    spca.fit(1, true_oos=False, stack_lags=False)
    for q in range(50):
        print("q", f'{q}/50')
        spca = GX_SPCA(chen_z_data_panel, tgt_factor, weak_test_start, N_factors=5)
        spca.fit(1, true_oos=True, stack_lags=False, quantile=q/50)
    print()

mkt
r2 vs ar model 0.9756477605683802
ar r2 vs mean model -0.008956470002243533
r2 vs mean model 0.9754296504664235
EV 0.9757113546722055
q 0/50
r2 vs mean model 0.910928196919435
EV 0.912132425870436
q 1/50
r2 vs mean model 0.9460886985323707
EV 0.9468175661174889
q 2/50
r2 vs mean model 0.9513907388758757
EV 0.952047924174791
q 3/50
r2 vs mean model 0.9521402703978741
EV 0.9527873222142814
q 4/50
r2 vs mean model 0.952190928914903
EV 0.9528372958405738
q 5/50
r2 vs mean model 0.9518456003336161
EV 0.9524966360170847
q 6/50
r2 vs mean model 0.9514305089742796
EV 0.95208715659121
q 7/50
r2 vs mean model 0.9502352793311875
EV 0.9509080861600069
q 8/50
r2 vs mean model 0.951367768372883
EV 0.95202526422737
q 9/50
r2 vs mean model 0.9508627011612251
EV 0.9515270254006493
q 10/50
r2 vs mean model 0.9513677260619672
EV 0.9520252224884874
q 11/50
r2 vs mean model 0.9522248300461618
EV 0.9528707386368451
q 12/50
r2 vs mean model 0.9524079706795473
EV 0.9530514032537452
q 13/50
r2 vs mean mode

r2 vs mean model 0.6322861099961867
EV 0.6284351370545197
q 20/50
r2 vs mean model 0.6364684648574397
EV 0.6326612926419377
q 21/50
r2 vs mean model 0.6495743659463514
EV 0.6459044484602507
q 22/50
r2 vs mean model 0.6506483703847707
EV 0.6469897006707668
q 23/50
r2 vs mean model 0.6499589652768532
EV 0.6462930756005671
q 24/50
r2 vs mean model 0.6511038859148519
EV 0.6474499866977711
q 25/50
r2 vs mean model 0.6512722933772591
EV 0.6476201578481937
q 26/50
r2 vs mean model 0.6501039352701317
EV 0.6464395638272783
q 27/50
r2 vs mean model 0.6485587559768062
EV 0.644878202268854
q 28/50
r2 vs mean model 0.6450403935820859
EV 0.6413229929701971
q 29/50
r2 vs mean model 0.6424875317439638
EV 0.6387433956670038
q 30/50
r2 vs mean model 0.6434102245601769
EV 0.6396757516076332
q 31/50
r2 vs mean model 0.6368255503272634
EV 0.6330221177759937
q 32/50
r2 vs mean model 0.631130660011556
EV 0.6272675863395463
q 33/50
r2 vs mean model 0.6295053491048448
EV 0.6256252539699703
q 34/50
r2 vs mean m

r2 vs mean model 0.5115162452845607
EV 0.5163352545855252
q 44/50
r2 vs mean model 0.5080107604837231
EV 0.5128643522323053
q 45/50
r2 vs mean model 0.509636314656863
EV 0.5144738699239778
q 46/50
r2 vs mean model 0.5359321422233863
EV 0.5405102828500011
q 47/50
r2 vs mean model 0.5341218490291552
EV 0.5387178486319514
q 48/50
r2 vs mean model 0.547544640360621
EV 0.5520082209102822
q 49/50
r2 vs mean model 0.5530208688221426
EV 0.5574304250657025

